In [1]:
import math
import numpy as np
import torch
import random
import matplotlib .pyplot as plt
%matplotlib inline

In [2]:
# Create Value class to store these expressions
class Value:
    # Initialize
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
        
    # Readable representation
    def __repr__(self):
        return f"Value(data={self.data})"

    # Overloading addition
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1
        
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
        
    # masking _backward
    def backward(self):
        # getting topological ordering
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # calling in the reversed topo order
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [3]:
class Neuron:

    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b) # sum xi*wi + self.b
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
 
    def __init__(self, nin, nout):   # nin - nr of inputs into a neuron; nout - nr of neurons in the layer
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:  # Multilayer Perceptron

    def __init__(self, nin, nouts):   # nin - nr of inputs into the MLP; nouts - list of layer sizes
        sz = [nin] + nouts  # gives all the layers including the input
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [4]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.19341746634883128)

In [5]:
xs = [
    [2.0, 3.0, -1.0],  # label: 1.0
    [3.0, -1.0, 0.5],  # label: -1.0
    [0.5, 1.0, 1.0],   # label: -1.0
    [1.0, 1.0, -1.0],  # label: 1.0
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [6]:
learning_rate = 0.05

for k in range(20):
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -learning_rate * p.grad
    
    print(k, loss.data)

0 4.124655618395419
1 3.797412992702845
2 3.668394864373145
3 3.4632092252456785
4 3.0876244264324266
5 2.39922921792908
6 1.5152672115388022
7 0.8802950422333563
8 0.5470364698987141
9 0.3700044350298335
10 0.2687565753583364
11 0.20617281844675073
12 0.16480683399052115
13 0.13593551075705637
14 0.11488495203825416
15 0.09898580576922195
16 0.08662670629886811
17 0.07678791381114669
18 0.06879765381610893
19 0.06219794297771143


In [7]:
ypred

[Value(data=0.8964061416261996),
 Value(data=-0.931933673092851),
 Value(data=-0.8306836027180352),
 Value(data=0.8652217070973022)]